In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load and preprocess data
df = pd.read_csv("data.csv")

# Select features
features = ['Voltage (V)','Current (A)','Battery Temperature (°C)','Environmental Temperature (°C)','Cycle Count','Battery Age (Days)','SOC (%)']
target = 'SOH (%)'

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(df[features])
y = df[target].values

# Create sequences for RNN (lookback of 3 time steps)
def create_sequences(X, y, time_steps=3):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X, y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Build RNN model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile and train
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, 
                   validation_split=0.2,
                   epochs=50, 
                   batch_size=32,
                   verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Plot results
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Battery Life')
plt.ylabel('Predicted Battery Life')
plt.title('Prediction vs Actual')

plt.tight_layout()
plt.show()

# Print evaluation metrics
print('Test MAE:', model.evaluate(X_test, y_test)[1])

ModuleNotFoundError: No module named 'tensorflow'